In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gc
from captum.attr import *
import quantus
import gc
import torchvision.transforms as transforms
import os
import torch.optim as optim
import torchvision.models as models
import sklearn
from sklearn.metrics import roc_auc_score

In [34]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a

        return y_pred

In [37]:
input_size=121
hidden_size=[256,256]
output_size=2
model = Network(input_size, hidden_size, output_size)
model.load_state_dict(torch.load("./model.pytorch"))
model = model.to(device)
model.eval()

Network(
  (fc1): Linear(in_features=121, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

In [38]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        #print(stdev)
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

# On FGSM

In [39]:
def compute_metrics_benign(train_loader, normal_model): 
    
    print("Computing metrics for {} for benign")

    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_gaussian4 = []
    attribution_gaussian5 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_gaussian4 = []
    logit_gaussian5 = []
    
    for step, (x_batch, y_batch) in enumerate(train_loader):
        
        
        images_adv, y_pred_adv = x_batch.to(device), y_batch.to(device)
        x_logits = normal_model(images_adv)
        
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.0005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.001)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        gaussian_noisy_images_4 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_4 = normal_model(gaussian_noisy_images_4)
        
        gaussian_noisy_images_5 = make_noise(images_adv, y_pred_adv, spread = 0.1)
        gaussian_logits_5 = normal_model(gaussian_noisy_images_5)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-gaussian_logits_4,p=1, dim=1) 
        diff5 = torch.norm(x_logits-gaussian_logits_5,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_gaussian4.extend(diff4.detach().cpu().numpy())
        logit_gaussian5.extend(diff5.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian4 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_4, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian5 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_5, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian4):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian4.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian5):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian5.append(c)
        
       
        if step%20==0:
            print(step)
        if step > 50:
            break
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_gaussian4,
        attribution_gaussian5,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
    logit_gaussian4,
    logit_gaussian5], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "Gaussian4 attribution",
        "Gaussian5 attribution",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "Gaussian4 logit robusntess",
        "Gaussian5 logit robusntess"
                    ])
            
    path = "kdd_Benign_eval.csv"
    df.to_csv(path)

In [40]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.data = self.data.drop(self.data.columns[0], axis=1)  # Remove unnecessary index column
        self.features = self.data.iloc[:, :-1].values  # Features (all columns except the last)
        self.labels = self.data.iloc[:, -1].values     # Labels (last column)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Load CSV file and split into train and test sets
dataset = CustomDataset('clean_examples.csv')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [41]:
compute_metrics_benign(test_loader, model)

Computing metrics for {} for benign
0
20
40


In [42]:
def compute_metrics_adv(train_loader, normal_model): 
    
    print("Computing metrics for {} for benign")

    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_gaussian4 = []
    attribution_gaussian5 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_gaussian4 = []
    logit_gaussian5 = []
    
    for step, (x_batch, y_batch) in enumerate(train_loader):
        
        
        images_adv, y_pred_adv = x_batch.to(device), y_batch.to(device)
        x_logits = normal_model(images_adv)
        
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.0005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.001)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        gaussian_noisy_images_4 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_4 = normal_model(gaussian_noisy_images_4)
        
        gaussian_noisy_images_5 = make_noise(images_adv, y_pred_adv, spread = 0.1)
        gaussian_logits_5 = normal_model(gaussian_noisy_images_5)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-gaussian_logits_4,p=1, dim=1) 
        diff5 = torch.norm(x_logits-gaussian_logits_5,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_gaussian4.extend(diff4.detach().cpu().numpy())
        logit_gaussian5.extend(diff5.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian4 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_4, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian5 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_5, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian4):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian4.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian5):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian5.append(c)
        
       
        if step%20==0:
            print(step)
        if step > 50:
            break
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_gaussian4,
        attribution_gaussian5,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
    logit_gaussian4,
    logit_gaussian5], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "Gaussian4 attribution",
        "Gaussian5 attribution",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "Gaussian4 logit robusntess",
        "Gaussian5 logit robusntess"
                    ])
            
    path = "kdd_Adv_eval.csv"
    df.to_csv(path)

In [49]:
k=[0.0015,0.0015,0.0015, 0.0015,0.0015,0.0015,0.0015,0.0015]
l=[1.07,1.00,0.8, 0.7, 0.6, 0.3,0.2,0.1]
m=[0.0,0.0,0.0, 0,0,0,0,0]
n=[100,40,25, 20, 18, 13, 7, 4]

In [47]:
def compute_TPR(adv1, a, b, adv2, c, d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2 in zip(adv1, adv2): 
        if value1<a or value1>b:
            TP += 1
        else:
            if value2<c or value2>d:
                TP+=1
            else: 
                FN+=1
    
    return (TP/(TP+FN))*100

In [48]:
def compute_FPR(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100

In [105]:
sc = []

compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)

score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [106]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [107]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [108]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)


advdataset = CustomDataset('adversarial_examples_FGSM4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [109]:
sc

[0.664674225876999, 0.6862745350967275, 0.6680753242019222, 0.670078893826213]

In [110]:
print(np.mean(sc), np.std(sc))

0.6722757447504655 0.008309917374894303


# PGD

In [111]:
sc= []

In [112]:
compute_metrics_benign(test_loader, model)
advdataset = CustomDataset('adversarial_examples_PGD1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [113]:
compute_metrics_benign(test_loader, model)
advdataset = CustomDataset('adversarial_examples_PGD2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [114]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_PGD3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [115]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_PGD4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [116]:
sc

[0.6835132435914226,
 0.6730930880983335,
 0.6850003088251608,
 0.6785664260157349]

In [117]:
print(np.mean(sc), np.std(sc))

0.680043266632663 0.004666343516592785


# BIM 

In [118]:
sc = []

In [119]:
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [120]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [121]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM3.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [122]:
# Load CSV file and split into train and test sets
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM4.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [123]:
sc

[0.6569768915313463, 0.6741421831040313, 0.669853231411419, 0.6666928194280315]

In [124]:
np.mean(sc), np.std(sc)

(0.6669162813687071, 0.006318242180779156)

# CW

In [125]:
sc = [] 
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW21.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [126]:
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW22.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [127]:
compute_metrics_benign(test_loader, model)


advdataset = CustomDataset('adversarial_examples_CW23.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [128]:
compute_metrics_benign(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW24.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

compute_metrics_adv(test_loader_adv, model)

df_ben = pd.read_csv("kdd_Benign_eval.csv")
attr_gaussian3 = df_ben.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3 = df_ben.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3), len(logit_gaussian3))
fpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
    fpr_results.append(FPR/100)
    
df_adv= pd.read_csv("kdd_Adv_eval.csv")
attr_gaussian3_eps1 = df_adv.iloc[0].values.flatten().tolist()[1:]
logit_gaussian3_eps1 = df_adv.iloc[5].values.flatten().tolist()[1:]
print(len(attr_gaussian3_eps1), len(logit_gaussian3_eps1)) 
tpr_results =[]
for t1,t2,t3,t4 in zip(k,l,m,n):
    TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

Computing metrics for {} for benign
0
20
40
Computing metrics for {} for benign
0
20
40
2567 2567
2567 2567


In [129]:
np.mean(sc), np.std(sc)

(0.7182395326860701, 0.007571592645126742)